In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
import requests
import subprocess
from dataclasses import dataclass
import xarray as xr
import cfgrib
import zipfile
import os
import shutil
import rasterio
from rasterio.transform import from_origin
from rasterio.transform import from_bounds
from rasterio.crs import CRS
#import h3
from pyproj import Transformer
import json
import pprint
import dask
import eccodes
import pygrib
import psycopg2
from osgeo import gdal
from collections import defaultdict
from scipy.spatial import KDTree
from datetime import datetime, timedelta
import rioxarray
from shapely.geometry import box
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
from metpy.units import units
from typing import Optional

os.chdir('/home/jeff/modelgen')

@dataclass
class DateTimeParts:
    year: int
    month: int
    day: int
    hour: int
    month_str: Optional[str] = None
    day_str: Optional[str] = None
    hour_str: Optional[str] = None
    date_str: Optional[str] = None

    @classmethod
    def from_datetime(cls, dt: datetime):
        month_str=str(dt.month).zfill(2)
        day_str = str(dt.day).zfill(2)
        hour_str=str(dt.hour).zfill(2)
        date_str = str(dt.year)+month_str+day_str
        return cls(year=dt.year, month=dt.month, day=dt.day, hour=dt.hour, month_str=month_str, day_str=day_str, hour_str=hour_str, date_str=date_str)
    
def windCalc(u,v):
        #print('windCalc Function')
        wind_abs = np.sqrt(u**2 + v**2)
        wind_dir_trig_to = np.arctan2(u/wind_abs, v/wind_abs)
        wind_dir_trig_to_degrees = wind_dir_trig_to * 180/np.pi ## -111.6 degrees
        wind_dir = wind_dir_trig_to_degrees + 180
        return wind_abs * 2.23694 #TO MPH
def K_to_F(temp):
    temp = ((temp - 273.15) * (9/5)) + 32
    return temp

def F_to_K(temp):
    temp = ((temp - 32) * 5/9) + 273.15
    return temp

In [ ]:
def getInitTime_GFS():
    mydate = (datetime.now())
    print(mydate)
    dateparts = DateTimeParts.from_datetime(mydate)

    if dateparts.hour >= 2 and dateparts.hour <= 8:
        hour_str = '00'
    elif dateparts.hour >= 9 and dateparts.hour <= 15:
        hour_str = '06'
    elif dateparts.hour >= 16 and dateparts.hour <= 21:
        hour_str = '12'
    elif dateparts.hour >= 22:
        hour_str = '18'
    elif dateparts.hour < 2:
        hour_str = '18'
        dateparts = DateTimeParts.from_datetime(mydate - dt.timedelta(days=1))
    else:
        print(f'warning: forecast hour {dateparts.hour} is not between 0 and 23')
        hour_str = None
    print(dateparts, hour_str)
    return dateparts, hour_str

getInitTime_GFS()

In [ ]:
dateparts, hour_str = getInitTime_GFS()
modelrun = datetime(year=dateparts.year,month=dateparts.month,day=dateparts.day,hour=dateparts.hour)
datetime_parts = DateTimeParts.from_datetime(modelrun)
datetime_parts
datetime_parts.date_str

In [ ]:
def getFilterGrib(runDate, indexHour, model='gfs-test'):
    index_list = []
    runTime = runDate.hour
    year = runDate.year
    month = runDate.month
    day = runDate.day
    fcsthr = indexHour
    runTime_str = str(runTime).zfill(2)
    fcsthr_str = str(fcsthr).zfill(3)
    runDate = rf'{year}{str(month).zfill(2)}{str(day).zfill(2)}'
    byte_ranges = defaultdict(list)

    if model == 'nbm':
        url_name = rf'https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend.{runDate}/{runTime_str}/core/blend.t{runTime_str}z.core.f{fcsthr_str}.co.grib2'
    elif model == 'gfs-test':
        url_name = rf'https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.{runDate}/{runTime_str}/atmos/gfs.t{runTime_str}z.pgrb2.0p25.f{fcsthr_str}'
    elif model == 'gfs-graph':
        url_name = rf'https://noaa-nws-graphcastgfs-pds.s3.amazonaws.com/graphcastgfs.{runDate}/{runTime_str}/forecasts_13_levels/graphcastgfs.t{runTime_str}z.pgrb2.0p25.f{fcsthr_str}'
    else:
        raise ValueError('Invalid model type')

    index_url = f"{url_name}.idx"
    print(f"Index URL: {index_url}")
    
    response = requests.get(index_url)
    index_content = response.text.splitlines()

# Corrected conditions dictionary with unique keys
    conditions = {
        'PRMSL': lambda param, level: param == 'PRMSL',
        'REFC': lambda param, level: param == 'REFC',
        'CSNOW': lambda param, level: param == 'CSNOW',
        'CFRZR': lambda param, level: param == 'CFRZR',
        'CICEP': lambda param, level: param == 'CICEP',
        'CFRZR': lambda param, level: param == 'CFRZR',
        'HGT_500mb': lambda param, level: param == 'HGT' and '500 mb' in level,
        'HGT_1000mb': lambda param, level: param == 'HGT' and '1000 mb' in level,
        'TMP_2m_above_ground': lambda param, level: param == 'TMP' and '2 m above ground' in level,
        'UGRD_10m_above_ground': lambda param, level: param == 'UGRD' and '10 m above ground' in level,
        'VGRD_10m_above_ground': lambda param, level: param == 'VGRD' and '10 m above ground' in level,
        'UGRD_100m_above_ground': lambda param, level: param == 'UGRD' and '100 m above ground' in level,
        'VGRD_100m_above_ground': lambda param, level: param == 'VGRD' and '100 m above ground' in level,
        'APCP': lambda param, level: param == 'APCP' and ((len(level) < 30) and (level.split(":"))[-1] == ''),
    }

    index_length = len(index_content)

    # Process each line in the index file
    for i in range(index_length):
        line = index_content[i]
        indexDict = line.split(":")
        startByte = indexDict[1]
        param = indexDict[3].strip()
        level = ":".join(indexDict[4:]).strip()

        # Check if the current line matches any condition
        matched = False
        for condition_name, condition_func in conditions.items():
            if condition_func(param, level):
                matched = True
                # Determine the end byte
                if i < index_length - 1:
                    next_line = index_content[i + 1]
                    next_indexDict = next_line.split(":")
                    endByte = next_indexDict[1]
                    byte_length = int(endByte) - int(startByte)
                else:
                    # Last line; we don't have a next start byte
                    byte_length = 0  # Or handle accordingly if you know the total file size

                # Append the byte range to the index list
                #index_list.append((int(startByte), int(endByte)))
                index_list.append(f'{startByte}-{endByte}') #USE FOR CURL
                #index_list.append((int(startByte), byte_length)) #USE FOR EARTKKIT.DATA
                print(f"Matched {param} {level}: Start byte {startByte}, Length {byte_length}")
                break  # No need to check other conditions once matched

    gribFile = f"data/gribs/{model.lower()}/latest/{model.lower()}-{fcsthr_str}.grb2"
    #gribFile = f"data/gribs/{model.lower()}/maxmin/{model.lower()}-{fcsthr_str}.grb2"

    if os.path.exists(gribFile):
        os.remove(gribFile)

    if len(index_list) > 0:
        for byte_range in index_list:
            print(f"Downloading byte range: {byte_range}")
            command = rf'curl --range {byte_range} {url_name} >> {gribFile}'
            os.system(command)
        else:
            print(f'no matches for forecast hour {fcsthr_str} ')

    #return gribFile
    return index_list, gribFile

In [ ]:
def appendGrib(model, result_list):
    #model = 'ndfd'
    latestGrb = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
    command_cp = f'cp {result_list[0]} {latestGrb}'
    subprocess.call(command_cp, shell=True)

    for grib_single in result_list[1:]:
        print(grib_single)
        command_append = f'wgrib2 -append {grib_single} -grib {latestGrb}'
        subprocess.call(command_append, shell=True)
    return pygrib.open(latestGrb)

#grbs = appendGrib(model, result_list)

In [ ]:
print(os.getcwd())
#os.chdir('/home/jeff/modelgen')
#print(os.getcwd())
dateparts, hour_str = getInitTime_GFS()
print(dateparts)
result_list = []
modelrunTimes = []
modelforecastTimes= []
modelforecastSteps = []
dir_root = ''
model = 'gfs-test'
for indexHour in range(30,31,6):
    #modelrun = datetime(dateparts.year,dateparts.month,dateparts.day,int(hour_str),0,0)
    modelrun = datetime(dateparts.year,dateparts.month,dateparts.day,12,0,0)
    datetime_parts = DateTimeParts.from_datetime(modelrun)
    print(datetime_parts)
    print('getFilterGrib')
    index_list, result = getFilterGrib(datetime_parts, indexHour, model)
    if os.path.exists(result):
        result_list.append(result)
        print('result', result)

latestGrb = f'{dir_root}data/gribs/{model.lower()}/latest/{model.lower()}-latest_test.grb2'
if os.path.exists(latestGrb):
    os.remove(latestGrb)


grbs = appendGrib(model, result_list)

In [ ]:
model='gfs-test'
testfile = f'data/gribs/{model.lower()}/latest/{model.lower()}-latest.grb2'
grbs = pygrib.open(testfile)
#grbs = grbs.select(name = '2 metre temperature')
for grb in grbs:
    print(grb)

In [ ]:
p6 = grbs[6]
pt = grbs[7]
print(p6)
print(pt)

print(pt.keys())
print(p6.timeIncrement)
print(pt.timeIncrement)
print(p6.lengthOfTimeRange)
print(pt.lengthOfTimeRange)
print(pt.stepType)
print(p6.stepRange)
print(pt.stepRange)


In [ ]:
step=24
stepRange = '18-24'
stepType = 'accum' #instant avg accum
#shortname = [['prmsl', 'refc']]
filter_kwargs = {'stepType' : f'{stepType}', 'stepRange': stepRange}
#filter_kwargs = {'stepType' : f'{stepType}', 'lengthOfTimeRange': 24}
#filter_kwargs = {'shortName' : f'{shortname}', 'step': step }
#filter_kwargs = {'step': step }
ds = xr.open_dataset(
    testfile,
    engine="cfgrib",
    backend_kwargs={"filter_by_keys": filter_kwargs})
ds=ds.sel(latitude = slice(57,20), longitude= slice(230,300))
print(ds)

In [ ]:
print(dir(ds['tp'].values))
print(ds['tp'].values.view())


In [ ]:
refc_snow = ds['refc'].where(ds['csnow'] == 1, other=0) + 100
refc_ice = ds['refc'].where((ds['cfrzr'] == 1) | (ds['cicep'] == 1), other=0) + 200

In [ ]:
print(refc_ice.max())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
#ax.contourf(ds['refc'].longitude, ds['refc'].latitude, refc_ice, transform=ccrs.PlateCarree(), levels=levels_ice, vmin=210, vmax=280, norm=norm_ice, cmap=cmap_ice, extend='neither')
contour3 = ax.contourf(ds['refc'].longitude, ds['refc'].latitude, ds['cfrzr'], transform=ccrs.PlateCarree())
ax.add_feature(cfeature.STATES)
ax.coastlines()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
#fig, ax = plt.subplots(figsize=(10, 8))
#contour = variable_F.plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap)
#contour = ax.contourf(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), levels = 32, vmin=-30, vmax=130, cmap=cmap)
#contour = ax.contour(ds['prmsl'].longitude, ds['prmsl'].latitude, ds['prmsl'].values)
contour2 = ax.contourf(ds['refc'].longitude, ds['refc'].latitude, ds['refc'].values, transform=ccrs.PlateCarree(), levels=levels, vmin=10, vmax=80, norm=norm, cmap=cmap, extend='neither')
contour3 = ax.contourf(ds['refc'].longitude, ds['refc'].latitude, refc_snow, transform=ccrs.PlateCarree(), levels=levels_snow, vmin=110, vmax=180, norm=norm_snow, cmap=cmap_snow, extend='neither')
contour4 = ax.contourf(ds['refc'].longitude, ds['refc'].latitude, refc_ice, transform=ccrs.PlateCarree(), levels=levels_ice, vmin=210, vmax=280, norm=norm_ice, cmap=cmap_ice, extend='neither')
contour = ax.contour(ds['prmsl'].longitude, ds['prmsl'].latitude, ds['prmsl'].values, transform=ccrs.PlateCarree(), colors='k', levels=plevels, linewidths=0.5)
ax.add_feature(cfeature.STATES)
ax.coastlines()
fig.colorbar(contour2, shrink=1.0, orientation='horizontal', pad=0.03, aspect=60, extend='both')

In [ ]:
testfile = f"data/gribs/{model}/latest/{model}-latest.grb2"

grbs = pygrib.open(testfile)
print(dir(grbs[1]))
for grb in grbs[12:13]:
    print(grb)
    stepType = grb.step
    print(stepType)
    fhour = grb.validDate
    print(fhour)
    tunits = grb.parameterUnits
    print(tunits)

In [ ]:
filter_kwargs = {'stepType' : f'{stepType}'}
grib_file = cfgrib.open_file(testfile, filter_by_keys={'stepType': 'max'})
# Print all metadata keys for the first message
print(dir(grib_file))
print(grib_file.attributes)

In [ ]:
model='gfs-graph'
testfile = f"data/gribs/{model}/latest/{model}-latest.grb2"
grbs = pygrib.open(testfile)
grb = grbs[2]
step = grb.step
print('step', step)
filter_kwargs = {'step': step}
ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs}
        )
print(ds)

In [ ]:
print(ds.attrs)

In [ ]:
testfile = "data/gribs/gfs/latest/gfs-latest.grb2"

grbs = pygrib.open(testfile)
print(dir(grbs[1]))
for grb in grbs[1:2]:
    #print(grb)
    print(grb.shortName, grb.typeOfLevel, grb.level)
    stepType = grb.stepType
    step = grb.step
    fhour = grb.validDate
    print('step', step)
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    print(ds)

In [ ]:
testfile = "data/gribs/gfs-graph/latest/gfs-graph-latest.grb2"

grbs = pygrib.open(testfile)
print(dir(grbs[1]))
for grb in grbs[2:4]:
    #print(grb)
    print(grb.shortName, grb.typeOfLevel, grb.level)
    stepType = grb.stepType
    step = grb.step
    fhour = grb.validDate
    print('step', step)
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs}
        )
    print(ds)

In [ ]:
datasets = cfgrib.open_datasets(testfile)

# Print out attributes for each dataset to see which filters can apply
for ds in datasets:
    print(ds)
    print(ds.attrs)  # Top-level attributes
    for var in ds.variables:
        print(f"Variable: {var}")
        print(ds[var].attrs) 

In [ ]:
import matplotlib.colors as mcolors

color_values = {
    -30: 'maroon',
    -20: 'teal',
    -10: 'lavender',
    0: 'white',
    10: 'fuchsia',
    20: 'purple',
    30: 'blue',
    40: 'aqua',
    50: 'green',
    60: 'yellow',
    70: 'orange',
    80: 'red',
    90: 'purple',
    100: 'white',
    110: 'pink',
    130: 'maroon'
}

# Create the normalized bounds (0 to 1) for the colormap
norm1 = mcolors.Normalize(vmin=min(color_values.keys()), vmax=max(color_values.keys()))

# Create a LinearSegmentedColormap
cmap = mcolors.LinearSegmentedColormap.from_list(
    'colormap_tempF',
    [(norm(value), color) for value, color in color_values.items()]
)

In [ ]:
import matplotlib.colors as mcolors

color_values_refc = {
    5: '#FFFFFF00',
    10: '#00FF0088',
    20: 'green',
    30: 'yellow',
    40: 'orange',
    50: 'red',
    60: 'purple',
    70: 'pink',
    80: 'orange'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc.keys()),
    ncolors=len(color_values_refc),
    extend='neither' )

values = list(color_values_refc.values())
levels = list(color_values_refc.keys())
print(levels)
print(values)
cmap = mcolors.ListedColormap(values)

norm = mcolors.BoundaryNorm(levels, ncolors=cmap.N, extend='neither')

color_values_refc_snow = {
    105: '#FFFFFF00',
    110: '#0000FF88',
    120: 'aqua',
    130: 'blue',
    140: 'purple',
    150: 'pink',
    160: 'white',
    170: 'green',
    180: 'orange'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm_snow = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc_snow.keys()),
    ncolors=len(color_values_refc_snow),
    extend='neither' )

values_snow = list(color_values_refc_snow.values())
levels_snow = list(color_values_refc_snow.keys())
print(levels_snow)
print(values_snow)
cmap_snow = mcolors.ListedColormap(values_snow)

norm_snow = mcolors.BoundaryNorm(levels_snow, ncolors=cmap_snow.N, extend='neither')

color_values_refc_ice = {
    205: '#FFFFFF00',
    210: '#FF000088',
    220: 'red',
    230: 'yellow',
    240: 'green',
    250: 'brown',
    260: 'white',
    270: 'blue',
    280: 'black'
}

# Create the normalized bounds (0 to 1) for the colormap
""" norm = mcolors.Normalize(vmin=min(color_values_refc.keys()), vmax=max(color_values_refc.keys()))
# Create a LinearSegmentedColormap
cmap_refc = mcolors.LinearSegmentedColormap.from_list(
    'colormap_refc',
    [(norm(value), color) for value, color in color_values_refc.items()]
) """

norm_ice = mcolors.BoundaryNorm(
    boundaries=list(color_values_refc_ice.keys()),
    ncolors=len(color_values_refc_ice),
    extend='neither' )

values_ice = list(color_values_refc_ice.values())
levels_ice = list(color_values_refc_ice.keys())
print(levels_ice)
print(values_ice)
cmap_ice = mcolors.ListedColormap(values_ice)

norm_ice = mcolors.BoundaryNorm(levels_ice, ncolors=cmap_ice.N, extend='neither')



In [ ]:
#cmap_name='gist_ncar'
cmap_name = 'colormap_tempF'
testfile = "data/gribs/gfs/latest/gfs-latest.grb2"
grbs = pygrib.open(testfile)
for grb in grbs[8:9]:
    shortname = grb.shortName
    longname = grb.name
    validDate = grb.validDate
    analDate = grb.analDate
    fcstHour = grb.forecastTime
    level = grb.level
    levtype = grb.levtype
    typeOfLevel = grb.typeOfLevel
    unit = grb.units
    cfName = grb.cfVarName
    stepType = grb.stepType
    step = grb.step
    p_units = grb.parameterUnits
    print('shortname', shortname)
    print('longname', longname)
    print('stepType', stepType)
    print('step', step)
    #step = timedelta(hours=fcstHour)
    filter_kwargs = {'stepType' : f'{stepType}', 'step': step}
    #filter_kwargs = {'shortName' : f'{shortname}', 'step': step }
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    ds=ds.sel(latitude = slice(57,20), longitude= slice(230,300))
    #ds = ds.sel(step=step)
    #print(ds)
    variables = list(ds.data_vars)
    variable = ds[variables[0]]
    variable = variable.rio.write_crs("EPSG:4326")
    variable.attrs['units'] = p_units
    variable = variable.metpy.quantify() 
    variable_F = variable.metpy.convert_units('degF')
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    #contour = variable_F.plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap)
    #contour = ax.contourf(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), levels = 32, vmin=-30, vmax=130, cmap=cmap)
    contour = ax.pcolormesh(variable.longitude, variable.latitude, variable_F.values, transform=ccrs.PlateCarree(), vmin=-30, vmax=130, cmap=cmap)
    ax.add_feature(cfeature.STATES)
    ax.coastlines()
    fig.colorbar(contour, shrink=1.0, orientation='horizontal', pad=0.03, aspect=60)
    #colorbar = contour.colorbar
    #colorbar.ax.set_position([0.78, 0.3, 0.03, 0.4])
    ax.set_extent([-130, -60, 20, 50])
    time_init = pd.Timestamp(ds.time.values).strftime('%D %H')
    time_valid = pd.Timestamp(ds.valid_time.values).strftime('%D %H')
    plt.title(f"GFS {shortname} deg F {time_init}Z fcst hr {step} {time_valid}Z")
    plt.savefig(f"data/images/gfs_plot/gfs_{shortname}_{str(step).zfill(3)}.png", bbox_inches='tight', pad_inches=0.1)

In [ ]:
print(dir(variable_F.metpy))
print(variable_F.metpy.unit_array)

In [ ]:
longname_list = []
shortname_list = []
analDate_list = []
validDate_list = []
typeOfLevel_list = []
fcstHour_list = []
level_list = []
levtype_list = []
filename_list = []
modelname_list = []
unit_list = []
bounding_box_wkt_list = []
testfile = "data/gribs/gfs/latest/gfs-latest.grb2"
model = 'gfs'
grbs = pygrib.open(testfile)
for grb in grbs:
    shortname = grb.shortName
    longname = grb.name
    validDate = grb.validDate
    analDate = grb.analDate
    fcstHour = grb.forecastTime
    level = grb.level
    levtype = grb.levtype
    typeOfLevel = grb.typeOfLevel
    modelname = model.upper()
    unit = grb.units
    cfName = grb.cfVarName
    stepType = grb.stepType
    step = timedelta(hours=fcstHour)

    print("shortname", shortname)
    print('fcstHour', fcstHour)


    #filter_kwargs = {"shortName": f"{shortname}", "typeOfLevel": f"{typeOfLevel}", "level": level}
    #filter_kwargs = {'shortName' : f'{shortname}', 'stepType' : f'{stepType}', 'forecastTime' : f'{fcstHour}'}
    filter_kwargs = {'shortName' : f'{shortname}'}
    ds = xr.open_dataset(
        testfile,
        engine="cfgrib",
        backend_kwargs={"filter_by_keys": filter_kwargs})
    ds = ds.sel(step=step)
    print(ds)
    variables = list(ds.data_vars)
    #print('xvars', variables)
    variable = ds[variables[0]]
    
    # HRRR variable = variable.rio.write_crs("+proj=lcc +lat_1=38.5 +lat_2=38.5 +lat_0=38.5 +lon_0=-97.5 +x_0=0 +y_0=0 +datum=WGS84 +units=m +a=6371229 +b=6371229")
    variable = variable.rio.write_crs("EPSG:4326")
    variable_3857 = variable.rio.reproject("EPSG:3857")
    bounds = variable.rio.bounds()
    bounding_box = box(bounds[0], bounds[1], bounds[2], bounds[3])
    bounding_box_wkt = bounding_box.wkt


    """ variable = variable.rio.reproject("EPSG:3857")
    # Get bounds after reprojection to EPSG:3857
    bounds = variable.rio.bounds()
    # Define EPSG:3857 valid bounds
    min_mercator, max_mercator = -20037508.34, 20037508.34
    # Clip the bounds to stay within EPSG:3857 valid range
    clipped_bounds = (
        min(max(bounds[0], min_mercator), max_mercator),  # minx
        min(max(bounds[1], min_mercator), max_mercator),  # miny
        min(max(bounds[2], min_mercator), max_mercator),  # maxx
        min(max(bounds[3], min_mercator), max_mercator)   # maxy
    )
    # Convert clipped bounds to a Shapely polygon
    bounding_box = box(clipped_bounds[0], clipped_bounds[1], clipped_bounds[2], clipped_bounds[3])
    # Convert to WKT format for PostGIS
    bounding_box_wkt = bounding_box.wkt """


    #write geotiff
    print('writing to raster', shortname, fcstHour)
    rootdir = 'data/images/gfs'
    rootdir_plot = 'data/images/gfs_plot'
    filename = f"{model}_{shortname}_{levtype}_{typeOfLevel}_{level}_{str(fcstHour).zfill(3)}"
    variable_3857.rio.to_raster(f"{rootdir}/{filename}.tiff")


    print('writing to png', shortname, fcstHour)
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    variable.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="viridis")  # Adjust colormap as needed
    #print('adding coastlines')
    #ax.coastlines()  # Add coastlines for geographical context
    #ax.gridlines(draw_labels=True)  # Add gridlines and labels if needed

    # Save as PNG
    plt.savefig(f"{rootdir_plot}/{filename}.png", bbox_inches='tight', dpi=300)
    plt.close()

    #append lists
    longname_list.append(longname)
    shortname_list.append(shortname)
    analDate_list.append(analDate)
    validDate_list.append(validDate)
    fcstHour_list.append(fcstHour)
    typeOfLevel_list.append(typeOfLevel)
    level_list.append(level)
    levtype_list.append(levtype)
    filename_list.append(filename)
    modelname_list.append(modelname)
    unit_list.append(unit)
    bounding_box_wkt_list.append(bounding_box_wkt)

In [ ]:
def connectDB(model='gfs', varname='wind'):
    #model = 'gfs'
    # Database connection details
    DB_HOST = 'localhost'
    DB_PORT = '5432'
    DB_NAME = 'geoserver_db'
    DB_USER = 'geoserver_user'
    DB_PASS = 'geoserver'

    # Path to the directory containing TIFF files
    TIFF_DIR = f'/usr/share/geoserver/data_dir/data/{model}_{varname}/'

    # Connect to the PostGIS database
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    ) 

    return conn

conn = connectDB()
print(conn)

In [ ]:
from psycopg2.extras import execute_values

conn = connectDB()


cur = conn.cursor()


#preprocessed_geom_list = [f"ST_GeomFromText('{wkt}', 3857)" for wkt in bounding_box_wkt_list]

# Create insertrows without including ST_GeomFromText in the values
insertrows = [
    (
        modelname, shortname, longname, validDate, analDate, fcstHour,
        level, levtype, typeOfLevel, unit, location, ingestion, the_geom
    )
    for modelname, shortname, longname, validDate, analDate, fcstHour,
        level, levtype, typeOfLevel, unit, location, ingestion, the_geom
    in zip(
        modelname_list, shortname_list, longname_list, validDate_list, analDate_list,
        fcstHour_list, level_list, levtype_list, typeOfLevel_list, unit_list,
        filename_list, validDate_list, bounding_box_wkt_list  # Pass WKT directly here
    )
]

# Define the SQL insert query with ON CONFLICT for upsert
insert_query = """
    INSERT INTO modeldata.meta_master (
        modelname, shortname, longname, validDate, analDate, fcstHour,
        level, levtype, typeOfLevel, unit, location, ingestion, the_geom
    ) VALUES %s
    ON CONFLICT (location) DO UPDATE SET
        modelname = EXCLUDED.modelname,
        shortname = EXCLUDED.shortname,
        longname = EXCLUDED.longname,
        validDate = EXCLUDED.validDate,
        analDate = EXCLUDED.analDate,
        fcstHour = EXCLUDED.fcstHour,
        level = EXCLUDED.level,
        levtype = EXCLUDED.levtype,
        typeOfLevel = EXCLUDED.typeOfLevel,
        unit = EXCLUDED.unit,
        ingestion = EXCLUDED.ingestion,
        the_geom = EXCLUDED.the_geom
"""

# Execute the bulk insert with execute_values, applying ST_GeomFromText in SQL
from psycopg2.extras import execute_values
execute_values(
    cur,
    insert_query,
    insertrows,
    template="(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, ST_GeomFromText(%s, 3857))",
    page_size=1000
)

# Commit the transaction
conn.commit()


# Close the cursor and connection
cur.close()
conn.close()


In [ ]:
plevels = np.linspace(88000,108000,51)
print(plevels)

In [ ]:
x = list(range(880,1080,4))
print(len(x))
for y in x:
    print(y)